## Pass Order to API

**Note:** Make sure you set your gateway id

The first command will write the items to DynamoDB the second will read

**Post order**

In [ ]:
import json, boto3, requests, datetime

#Set gateway id
gwid = ''

x = datetime.datetime.now()
orderid = '151'
accountid = 'p150'
vendorid = 'Coffee One'
orderdate = str(x)
coffeetype = 'Short Black'
coffeesize = 'Small'
unitprice = '4.50'
quantity = '1'

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

response = requests.post(url,json={'order':{
            'orderid': orderid,
            'accountid': accountid,
            'vendorid': vendorid,
            'orderdate':orderdate,
            'details':{
                'coffeetype': coffeetype,
                'coffeesize': coffeesize,
                'unitprice': unitprice,
                'quantity': quantity
            }
        }
    })
print(response)

**Get Order**

In [ ]:
response_get = requests.get(url, params={'orderid':150})

print(response_get.json())